In [2]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
import re
import os
import csv
import time
import codecs

In [3]:
class TagCrawller:

    #크롤러 객체 초기화
    def __init__(self, chromedriver_path, original_file_path, url_csv_filepath, tag_csv_filepath, c_num, total_c_num):
        self.driver = webdriver.Chrome(chromedriver_path)
        self.original_file_path = original_file_path
        self.url_csv_filepath = url_csv_filepath
        self.tag_csv_filepath = tag_csv_filepath
        self.c_num = c_num
        self.total_c_num = total_c_num
        self.CNT = 0
   
    def make_indi_file(self):
        f = open(self.original_file_path, 'r', encoding='utf-8')
        url_list = csv.reader(f)
        
        total_line_cnt = 0 # 행수
        total_val_cnt = 0 # 총 url 수
        
        for line in url_list:
            total_line_cnt+=1
            for value in line:
                total_val_cnt+=1
                
        f.seek(0)
        
        share_val = total_line_cnt // self.total_c_num
        share_tot_val = total_val_cnt // self.total_c_num
        
        start_num = share_val * (self.c_num-1)
        self.start_num = share_tot_val * (self.c_num-1)
        
        if self.c_num<self.total_c_num:
            end_num = share_val * self.c_num - 1
            self.end_num = share_tot_val * self.c_num - 1
        
        else:
            end_num = total_line_cnt - 1
            self.end_num = total_val_cnt - 1
        
        cnt = 0
        for line in url_list:
            if cnt < start_num:
                cnt+=1
                continue
                    
            if cnt == end_num:
                break
                
            self.write_csv_url(line)
            cnt+=1
        
        f.close()
                
                
    #크롤러 객체 준비
    def crawlling_ready(self):
        self.make_indi_file()
        self.read_csv()

    # url크롤링이 완료된 후 수집된 url을 방문하며 
    # 유저 ID , 포스팅 날짜, 해시태그 수집
    def info_extractor(self,url):
        self.driver.implicitly_wait(2)
        self.driver.get(url)
        source = self.driver.page_source 
        soup = BeautifulSoup(source, 'lxml') 
        
        try:
            hashtag_list = soup.findAll("a", href=lambda value: value and value.startswith("/explore/tags"))
            posting_date = soup.find("time")['title'] 
            userId = self.driver.find_element_by_xpath("//*[starts-with(@class, 'FPmhX')]").text
            
            if len(hashtag_list)==0:
                self.CNT+=1
                return print("해쉬 없음!")
        
        except NoSuchElementException:  
            return print("NoSuchElementException!")
        
        except StaleElementReferenceException:
            return print("StaleElementReferenceException!")
        
        except TypeError:
            self.CNT+=1
            return print("페이지 없음!")
       
        h_list = self.tag_generator(hashtag_list)
        hash_list = []
        
        for item in h_list:
            hash_list.append(item)
        
        # 추후에 중복 처리를 위한 데이터
        overall = userId + posting_date + str(hash_list) 
        
        total_list = []
        total_list.append(userId)
        total_list.append(posting_date)
        total_list.append(hash_list)
        total_list.append(overall)
        
        self.write_csv_tag(total_list)
        
    
    #url 을 저장하기 위한 csv 파일을 만든다     
    def write_csv_url(self,list):
        with codecs.open(self.url_csv_filepath,"a","utf-8") as fp:
            writer=csv.writer(fp,delimiter=",")
            writer.writerow(list)
     
    #id, date, tag를 저장하기 위한 csv 파일을 만든다     
    def write_csv_tag(self,list):
        with codecs.open(self.tag_csv_filepath,"a","utf-8") as fp:
            writer=csv.writer(fp,delimiter=",")
            writer.writerow(list)
    
    def tag_generator(self,hashtag_list):
        for item in hashtag_list:
            yield item.getText()
    
    def url_generator(self,url_list):    
        for line in url_list:
            for href in line:
                url = 'https://www.instagram.com'+href
                yield url
    
    # url 목록이 저장된 csv 파일을 읽으며 해당 url에 방문하여 크롤링한다
    def read_csv(self):
        print("-------------------%d번 크롤러 시작 : %d ~ %d---------------------" %(self.c_num, self.start_num, self.end_num))
        #f = open(self.url_csv_filepath, 'r', encoding='utf-8')
        with open(self.url_csv_filepath, 'r', encoding='utf-8') as f:
            url_list = csv.reader(f)
            self.CNT = self.start_num
            
            for url in self.url_generator(url_list):
                print("%d번..." %self.CNT)

                # url이 공백일 경우 예외처리
                if url=="":
                    continue

                self.info_extractor(url)
                self.CNT+=1
            
            print("--------------TAG 크롤링 끝!----------------")
            self.driver.close()
        
        #f.close()

In [5]:
keyword = input("크롤링할 키워드? : ") # url 해시태그
orginal_file_path = 'C:/Users/acorn/weather_bigdata/original_url/url_'+keyword+'.csv'
chromedriver_path = 'C:/Users/acorn/Desktop/weather/chromedriver_win32/chromedriver'

total_c_num = int(input("크롤러 총 몇 개? : "))
c_num = int(input("몇 번 크롤러? : "))

url_csv_filepath = 'C:/Users/acorn/url_'+keyword+str(c_num)+'.csv'
tag_csv_filepath = 'C:/Users/acorn/tag_'+keyword+str(c_num)+'.csv'

crawller = TagCrawller(chromedriver_path, orginal_file_path, url_csv_filepath, tag_csv_filepath, c_num, total_c_num)
crawller.crawlling_ready()

크롤링할 키워드? : 제주도
크롤러 총 몇 개? : 3
몇 번 크롤러? : 1
-------------------1번 크롤러 시작 : 0 ~ 27862---------------------
0번...
1번...
2번...
3번...
페이지 없음!
5번...
6번...
7번...
8번...
9번...
10번...
11번...
12번...
13번...
14번...
페이지 없음!
16번...
17번...
18번...
19번...
20번...
페이지 없음!
22번...
페이지 없음!
24번...
페이지 없음!
26번...
27번...
페이지 없음!
29번...
30번...
31번...
페이지 없음!
33번...
34번...
35번...
36번...
37번...
38번...
39번...
40번...


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 6.1.7601 SP1 x86_64)
